In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import sys
import pyemu
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")

# Introduction
"EVA" stands for "ensemble variance analysis" and “DSI” stands for “data space inversion”. EVA enables the exploration of a model prediction's posterior distribution without requiring the exploration of the posterior distribution of model parameters. In other words, using a few model runs from a prior Monte Carlo run, forecast uncertainty is estimated both before (prior) and after (posterior) incorporating the observation (or potential observation) dataset through history matching. But, the history matching isn't required - this is an approximation. Starting to sound like FOSM? The maths and ideas are very similar! We are just able to skip the expensive Jacobian calculation in this approach. An extension of this is to treat the EVA surrogate model as if it were a process model, define "parameters" that control its behavior, and then approximate history matching using the surrogate to evaluate potential improvement to forecast calculations. That extension is DSI. 

So how does it work? This is achieved by constructing a surrogate model using principal component analysis (PCA) of the covariance matrix of model outputs (i.e., observations and forecasts). This matrix links model outputs corresponding to field measurements with predictions of interest. The resulting predictions are then conditioned on real-world measurements of system behavior.

The main steps are:
1. Generate an ensemble of model outputs (both historical measurements and forecast quantities) simulated using with a parameter ensemble, usually the prior in a Bayesian sense. 
2. Construct a surrogate/data-driven model from the covariance between historical and forecast model outputs.
3. Condition/train the surrogate model with measured data.
3. retrieve the emulated forecasts' posterior probability distribution.

The following notebook goes through the the method described by [Sun and Durlofsky (2017)](https://doi.org/10.1007/s11004-016-9672-8) and [Lima et al (2020)](https://doi.org/10.1007/s10596-020-09933-w). The GMDSI Youtube channel also has an excellent [overview of the method by John Doherty](https://youtu.be/s2g3HaJa1Wk?si=fzRd0WQTtK7WaeH6). Although we use different notation, the approach described herein is similar to the "Option 2" discussed in John's Youtube video.


# Generate the ensemble of model outputs

First we need some "training data". Let us start by cooking up some fake "model outputs". Say we have a "model" that outputs three "measured" observations and a "prediction". Let us say we have run our model with a 1000 samples (i.e., realizations) of the prior. 

In [ ]:
# Mean values for each variable
mean = [0, 1, 2, 3]

# Covariance matrix: answer at the back of the book...
true_cov = [
    [1, 0.8, 0.5, 0.5],  
    [0.8, 1, 0.3, 0.3],  
    [0.5, 0.3, 1, .2],   
    [0.5, 0.3,.2,1]
]

# Number of samples to generate a.k.a. ensemble size
nreal = 1000

# Generate the fake prior observation ensemble - normally this would require running the model
# for each realization...
np.random.seed(42)
fake_sim_ensemble = pd.DataFrame(np.random.multivariate_normal(mean, true_cov, nreal),
                                 columns=["prediction","obs1","obs2","obs3"])
fake_sim_ensemble.head()

In [ ]:
_ = pd.plotting.scatter_matrix(fake_sim_ensemble)

## Calculate the covariance matrix $\mathbf{C}_d^{1/2}$

We will need to use the empirical Covariance matrix 

$\mathbf{C}_d$ is calculated as:

$$
\mathbf{C}_d = \frac{1}{N-1} \sum_{i=1}^{N} (\mathbf{d}_i - \bar{\mathbf{d}}) (\mathbf{d}_i - \bar{\mathbf{d}})^T
$$

where $N$ is the number of samples in the ensemble, $\mathbf{d}_i$ is the $i$-th sample of the ensemble, and $\bar{\mathbf{d}}$ is the mean of the ensemble.
Which is equivalent to:

$$
\mathbf{C}_d = \Delta\mathbf{D} \Delta\mathbf{D}^T
$$

where $\Delta\mathbf{D}$ is the matrix of the ensemble of model outputs with the mean subtracted from each row (also referred to as a deviations matrix), and is calculated as:

$$
\Delta \mathbf{D} = \frac{1}{\sqrt{N_e - 1}} \left[ \mathbf{d}_1 - \bar{\mathbf{d}}, \ldots, \mathbf{d}_{N_e} - \bar{\mathbf{d}} \right].
$$

where $N_e$ is the number of samples in the ensemble.



# Ensemble Variance Analysis 
EVA uses the same maths as FOSM to propagate uncertainty from a history-matching dataset onto forecasts, where both quantities are simulation outputs from the same model. At the heart of this is the Schur complement (see notebook on FOSM). We can use this relationship to calculate propagation of variance from observations to forecasts using partitions of the covariance matrix $\mathbf{C}_d$. 

In general, the Schur complement propagates covariance from one quantity (in our case, history-matching observations) to another (in our case, forecasts). The formula in general is:  
$$
\tilde{\mathbf{C}}_{22}=\mathbf{C}_{22}-\mathbf{C}_{21}\mathbf{C}_{11}^{-1}\mathbf{C}_{12}
$$

For this to work in our case, then, $\tilde{\mathbf{C}}_{22}$ is the updated covariance of forecasts, $\mathbf{C}_{22}$ is the prior covariance of forecasts, $\mathbf{C}_{11}$ is the covariance of observation data, and $\mathbf{C}_{21}=\mathbf{C}_{12}$ is the cross-covariance between observations and forecasts. 

All of these covariance matrices are partitions of $\mathbf{C}_d$.  

For a single forecast of interest, $\mathbf{C}_{22}$ reduces to a single diagonal element representing the forecast variance, so we can restate the equation as
$$
\tilde{\sigma}_2=\sigma_{2}-\mathbf{C}_{21}\mathbf{C}_{11}^{-1}\mathbf{C}_{12}
$$

# Moving on to Data-space inversion

Given this relationship, we can also take this covariance between history-matching observations and forecasts further, regarding it as a surrogate model. Following the notation in [Lima et al (2020)](https://doi.org/10.1007/s10596-020-09933-w), $\mathbf{d}$ is the vector of model simulated outputs that correspond to both predictions and measurements. As mentioned above, the main idea behind the method is to use principle components analysis (PCA) to write the vector of predicted data ($\mathbf{d}_{\text{PCA}}$) as:

$$
\mathbf{d}_{\text{PCA}} = \bar{\mathbf{d}} + \mathbf{C}_d^{1/2} \mathbf{x}
$$

in which $\bar{\mathbf{d}}$ and $\mathbf{C}_d$ are the mean and the covariance matrix of  $\mathbf{d}$, and $\mathbf{x}$ is a vector of random numbers. Both of which are obtained from the ensemble of model outputs.  

>_A note on $\mathbf{x}$...This vector contains "parameters" for the surrogate model. Importantly, these are not the base parameters of the underlying model, but you can think of them as a mapping from base parameters to "super parameters" that drive the surrogate model. They are derived from the PCA analysis and we go into more detail below._

## Calculate the mean-vector $\bar{\mathbf{d}}$

In [ ]:
# the mean
d_bar = fake_sim_ensemble.mean()
d_bar

In [ ]:
# note that this is an approximation of the `true_cov` matrix
Cd = fake_sim_ensemble.cov()
Cd

In [ ]:
# NOTE: to maintain consistency with notation used in the papers, 
# here we need to transpose our ensemble to be of shape (nobs,nreal)
deltaD = fake_sim_ensemble.T.apply(lambda x: (x - x.mean()) / np.sqrt(fake_sim_ensemble.shape[0]-1),axis=1)
deltaD.head()

Let's calculate $\Delta \mathbf{D}$.

Why are we talking about $\Delta\mathbf{D}$? Because $\mathbf{C}_d^{1/2}$, used in the first equation we presented, is calculated using the singular value decomposition (SVD) of $\Delta\mathbf{D}$ (See the intro to SVD notebook!):

$$
\Delta\mathbf{D} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T
$$

where $\mathbf{U}$ and $\mathbf{V}$ are orthogonal matrices forming a basis for $\Delta\mathbf{D}$ and $\mathbf{\Sigma}$ is a diagonal matrix with the singular values of $\Delta\mathbf{D}$.


In [ ]:
# SVD
U, Sigma, Vt = np.linalg.svd(deltaD, full_matrices=False)
U.shape,Sigma.shape,Vt.shape



From these, we can now calculate the square root of $\mathbf{C}_d$ as:

$$
\mathbf{C}_d^{1/2} = \mathbf{U} \mathbf{\Sigma}
$$

where $\mathbf{\Sigma}^{1/2}$ is a diagonal matrix with the square root of the singular values of $\Delta\mathbf{D}$.

In [ ]:
Cd_sqrt = np.dot(U,np.diag(Sigma))

## The model emulator

In DSI, the emulator is nothing more than a linear transformation of the model outputs. The emulator is constructed by projecting the model output ensemble onto the principal components of the covariance matrix of the model outputs. 

The model emulator is "run" by calculating $\bar{\mathbf{d}} + \mathbf{C}_d^{1/2} \mathbf{x}$, which is super fast compared to executing a process-based numerical model. The vector $\mathbf{x}$ are referred to as "latent-space parameters" and are so-called "standard-normal" numbers: independent random numbers with mean of zero and variance of 1.0.  Within the DSI history matching workflow, the values of $\mathbf{x}$ will be "PEST adjustable parameters"

In [ ]:
# the "prior" mean of emulator "parameters" i.e. the PCA latent-space parameters
x = np.zeros_like(Sigma)
x

So, to execute a "forward run", we just calculate:
$$
\mathbf{d}_{\text{PCA}} = \bar{\mathbf{d}} + \mathbf{C}_d^{1/2} \mathbf{x}
$$

In [ ]:
# a model-emulator "forward run"
d_bar.values + np.dot(Cd_sqrt,x), d_bar.values

### But wait, we haven't done anything?
We are starting with these PCA-related "latent-space parameters" all as 0, so we get a trivial result when we run the forward model. But...this formulation gives us an opportunity to "map" from the mean of the observations to new ones, if we just "calibrate" those parameters in a meaningful way. It's like starting with a forward model and a set of parameters with unknown values. If we perform calibration with real observation values, we can learn meaningful values for $\mathbf{x}$. Let's try that!

### Dummy calibration

In practice, how do we handle this with PEST? The $\bar{\mathbf{d}}$ and $\mathbf{C}_d^{1/2}$ matrices are constructed and recorded in the PEST model directory. Then, a forward run script is prepared which reads these matrices, as well as the PEST-adjusted values of the vector $\mathbf{x}$, and calculates the model emulator outputs. 

Let's demonstrate this with a simple example. Here is what a forward run might look like:

In [ ]:
def forward_run(x):
    #pretend to read d_bar
    #pretend to read Cd_sqrt
    #pretend to read x
    return d_bar.values + np.dot(Cd_sqrt, x)

x = np.zeros_like(Sigma)
obs = forward_run(x)
obs

And now lets choose a "truth":

In [ ]:
# choose a realisationas the truth
truth = fake_sim_ensemble.iloc[-1]
truth

...and now calibrate the emulator to the truth observations (don't do this at home folks...this only works well because it is a super simple example):

In [ ]:
# find the pvals that minimize the difference between the truth and the forward run
from scipy.optimize import minimize

def objective(x):
    # objective does not include the prediction column
    return np.sum((forward_run(x)[1:] - truth[1:])**2)

# intial parameters
pvals_initial = np.zeros_like(Sigma)
initial_outputs = forward_run(pvals_initial)

# optimize
res = minimize(objective, pvals_initial,tol=1e-8)
assert res.success, "failed to find optimal solution"
res

In [ ]:
final_pvals = res.x
final_outputs = forward_run(final_pvals)

Let's plot the initial and final historic and forecast values:

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
ax.scatter(truth[1:],initial_outputs[1:],marker='o',s=10,c="0.5",label="initial obs values")
ax.scatter(truth[1:],final_outputs[1:],marker='o',s=10,c="b",label="final obs values")
ax.scatter(truth[0],initial_outputs[0],marker='^',s=20,c="0.5",label="initial forecast")
ax.scatter(truth[0],final_outputs[0],marker='^',s=20,c="b",label="final forecast")


mn = min(ax.get_ylim()[0],ax.get_xlim()[0])
mx = max(ax.get_ylim()[1],ax.get_xlim()[1])
ax.plot([mn,mx],[mn,mx],"k--",alpha=0.5)
ax.set_xlim(mn,mx)
ax.set_ylim(mn,mx)
ax.legend(loc="upper left")


Nailed it!

# Rejection sampling with DSI

DSI is so efficient, we can try to do rejection sampling.  So first, we need to generate a lot of (latent-space) parameter sets, then run them through the DSI emulator.  Then we can filter out output sets that don't reproduce the historic observations "well enough"...here we go

In [ ]:
num_reals = 10000
prior_latentpar_ensemble = np.random.standard_normal((num_reals,Sigma.shape[0]))
prior_latentpar_ensemble.shape

In [ ]:
prior_df = pd.DataFrame([forward_run(vec) for vec in prior_latentpar_ensemble],columns=fake_sim_ensemble.columns)
prior_df.head()

Ok so now we need to decide which results are "good enough" and which ones aren't.  In the PEST world, this usually done with weighted sum-of-squared residual, so lets do that (assuming weights of 1.0 for each observation value):

In [ ]:
prior_df["phi"] = prior_df.apply(lambda x: ((x.values[1:]-truth[1:])**2).sum(),axis=1) 

In [ ]:
_ = plt.hist(prior_df.phi.values,facecolor="0.5",alpha=0.5)

If we are being strict and proper, we should only accept realizations that have a phi less than or equal to nobs

In [ ]:
nobs = fake_sim_ensemble.shape[1] - 1
post_df = prior_df.loc[prior_df.phi<=nobs,:]
post_df.shape[0], "posterior reals"

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(10,10))
ax = axes[0]
_ = ax.hist(prior_df.phi,facecolor="0.5",alpha=0.5,density=True,label="prior")
_ = ax.hist(post_df.phi,facecolor="b",alpha=0.5,density=True,label="posterior")
ylim = ax.get_ylim()
ax.plot([nobs,nobs],ylim,"k--",lw=3,label="behavorial threshold")
ax.legend(loc="upper right")
ax.set_title("phi",loc="left")
ax = axes[1]
_ = ax.hist(prior_df.prediction,facecolor="0.5",alpha=0.5,density=True,label="prior")
_ = ax.hist(post_df.prediction,facecolor="b",alpha=0.5,density=True,label="posterior")
ylim = ax.get_ylim()
ax.plot([truth[0],truth[0]],ylim,"r",lw=3,label="truth")
ax.legend(loc="upper left")
ax.set_title("prediction",loc="left")

Boo ya!  thats pretty awesome - pure Bayesian sampling...and it worked (in that we captured the truth with the posterior)!  #winning